# Filtering JetStream Events

In this notebook, we'll filter events from the Redis Stream that we created in the previous notebook. We'll use a combination of techniques to filter the events:

1. Deduplication using Redis Bloom Filter to avoid processing the same event multiple times
2. Content-based filtering using a machine learning model to identify software-related posts
3. Storing filtered events in Redis for further processing

Redis Bloom Filter is a probabilistic data structure that allows us to check if an element is in a set. It's very memory efficient and has a constant time complexity for both insertion and lookup operations. The trade-off is that it can have false positives, but the probability of false positives can be controlled by the size of the filter.

Machine learning models can be used to classify text into different categories. In this notebook, we'll use a pre-trained zero-shot classification model to classify posts as software-related or not.

## Consuming from Redis Streams

### Model Redis Streams Event
In this section, we'll define a data class to represent the events stored in the Redis Stream. This model will be used to deserialize the events from the stream.

### Creating a Redis Client
Create a Jedis client to connect to Redis. This is a reusable client that can be used to interact with Redis Streams.

### Creating a Consumer Group
Create a consumer group to read from the Redis Stream. A consumer group allows multiple consumers to read from the same stream without duplicating the work. Each consumer in the group will receive a different subset of the messages.

A consumer group can be created in Redis with the XGROUP CREATE command:

`XGROUP CREATE streamName groupName id [MKSTREAM]`

To create a consumer group in this notebook, we will encapsulate the command in a function. The function will take the stream name and the group name as parameters.

In [24]:
%use coroutines

In [25]:
import redis.clients.jedis.StreamEntryID
import dev.raphaeldelio.*

fun createConsumerGroup(streamName: String, consumerGroupName: String) {
    try {
        jedisPooled.xgroupCreate(streamName, consumerGroupName, StreamEntryID("0-0"), true)
    } catch (_: Exception) {
        println("Group already exists")
    }
}

In [26]:
createConsumerGroup("jetstream", "printer-example")

### Reading from the Stream
Create a reusable function to read from the stream. This function will read from the stream and return a list of entries. It uses the XREADGROUP command to read from the stream as part of a consumer group:

`XREADGROUP GROUP groupName consumerName COUNT count BLOCK blockTime streamName id`

The command will be encapsulated in a function that takes the stream name, consumer group name, consumer name, and count as parameters. The function will return a list of entries.

In [27]:
import redis.clients.jedis.params.XReadGroupParams
import redis.clients.jedis.resps.StreamEntry

fun readFromStream(
    streamName: String,
    consumerGroup: String,
    consumer: String, count: Int
): List<Map.Entry<String, List<StreamEntry>>> {
    return jedisPooled.xreadGroup(
        consumerGroup,
        consumer,
        XReadGroupParams().count(count),
        mapOf(
            streamName to StreamEntryID.XREADGROUP_UNDELIVERED_ENTRY
        )
    ) ?: emptyList()
}

### Acknowledging Messages
Create a function to acknowledge the message. This is important to let Redis know that the message has been processed successfully, so it won't be delivered to other consumers in the group.

This is done by using the XACK command:

`XACK streamName groupName id`

The command will be encapsulated in a lambda function that takes the stream name, consumer group name, and entry as parameters. The function will acknowledge the message by calling the XACK command.

In [28]:
fun ackFunction(): (String, String, StreamEntry) -> Unit {
    return { streamName, consumerGroup, entry ->
        jedisPooled.xack(
            streamName,
            consumerGroup,
            entry.id
        )
    }
}

### Consuming the Stream
Create a reusable function to consume the stream.

This function implements a pipeline pattern where each event is processed sequentially by a series of handlers. If any handler returns false, the processing stops for that event.

After processing the event, the function acknowledges the message using the ack function.

In [29]:
import kotlinx.coroutines.*

fun consumeStream(
    streamName: String,
    consumerGroup: String,
    consumer: String,
    handlers: List<(Event) -> Pair<Boolean, String>>,
    ackFunction: ((String, String, StreamEntry) -> Unit),
    count: Int = 1,
    limit: Int = 5
) {
    var lastMessageTime = System.currentTimeMillis()
    var consumed = 0

    while (consumed < limit) {
        val entries = readFromStream(streamName, consumerGroup, consumer, count)
        val allEntries = entries.flatMap { it.value }
        allEntries.map { entry ->
            consumed++
            val event = Event.fromMap(entry)

            for (handler in handlers) {
                val (shouldContinue, message) = handler(event)
                ackFunction(streamName, consumerGroup, entry)

                if (!shouldContinue) {
                    println("$consumer: Handler stopped processing: $message")
                    break
                }
            }
        }

        if (allEntries.isEmpty()) {
            val now = System.currentTimeMillis()
            if (now - lastMessageTime >= 2_000) {
                println("$consumer: No new messages for 2 seconds. Stopping.")
                break
            }
        }
    }

}

To test the consumeStream function, we'll create a simple handler that prints the event's URI.

In [30]:
val printUri: (Event) -> Pair<Boolean, String> = {
    println("Got event from ${it.uri}")
    Pair(true, "OK")
}

In [31]:
runBlocking {
    consumeStream(
        streamName = "jetstream",
        consumerGroup = "printer-example",
        consumer ="printer-1",
        handlers = listOf(printUri),
        ackFunction = ackFunction(),
        count = 1,
        limit = 100
    )
}

Got event from at://did:plc:dljiwdbyrlednpts5cfrdxjd/app.bsky.feed.post/3lprr6pibsc2q
Got event from at://did:plc:gwb2rljvrhfthwhc4npm5fqv/app.bsky.feed.post/3lprr6rgkxk2s
Got event from at://did:plc:mrhmeo6mwwiuenc4p52362je/app.bsky.feed.post/3lprr6plq5k2p
Got event from at://did:plc:vdual2pry3a2twyts7lugcdz/app.bsky.feed.post/3lpopt7zjhc2k
Got event from at://did:plc:7go7clbyqlfzjt7fowl7mvzs/app.bsky.feed.post/3lprr6pjls22u
Got event from at://did:plc:rlbwwm4iqolivzsnrprxu5si/app.bsky.feed.post/3lprr6pf57k2c
Got event from at://did:plc:gbqftnsx2huhgsueubrrjwoh/app.bsky.feed.post/3lprr6oz3hs25
Got event from at://did:plc:sm4dazdbzru7arfloasfybsy/app.bsky.feed.post/3lprr6po7as2z
Got event from at://did:plc:izp6rua2lw7ffodg2s77n463/app.bsky.feed.post/3lprr6ptxt22w
Got event from at://did:plc:tssxcazq3tmfehwm6e3namza/app.bsky.feed.post/3lprr6pfxlc24
Got event from at://did:plc:xemibbns6eesvaoxt3vso3lc/app.bsky.feed.post/3lprr6q3c722k
Got event from at://did:plc:irw5ek7trxl33tqh4mxqncxh/a

## Semantic Classifier
In this section, we'll use a vector similarity search to filter posts based on their content.
We'll use a embed a list of references and classify the posts based on their similarity to the references.
If the post is similar to the references, it will be classified as artificial-intelligence-related.

### Including Spring AI Redis Vector Store
The Redis Vector Store is a Spring AI module that allows you to store and retrieve vectors in Redis. It uses the Redis Vector Search feature to perform similarity searches on the vectors.

In [32]:
@file:DependsOn("org.springframework.ai:spring-ai-redis-store:1.0.0-RC1")

### Initializing an embedding model
The embedding model is used to convert text into vectors. The Redis Vector Store uses the embedding model to store and retrieve vectors in Redis.

The Transformers Embedding Model provided by Spring AI uses all-minilm-l6-v2 model by default which has 384 dimensions.

The model is loaded from the Hugging Face Hub and can be used to convert text into vectors.


In [33]:
import org.springframework.ai.transformers.TransformersEmbeddingModel

val embeddingModel = TransformersEmbeddingModel()
embeddingModel.afterPropertiesSet()

### Creating the store (and the index in Redis)
By creating an index in Redis, we can use the Redis Vector Search feature to perform similarity searches on the vectors.


In [34]:
import org.springframework.ai.vectorstore.redis.RedisVectorStore
import org.springframework.ai.vectorstore.redis.RedisVectorStore.MetadataField
import redis.clients.jedis.search.Schema.FieldType
import dev.raphaeldelio.jedisPooled

val redisVectorStore = RedisVectorStore.builder(jedisPooled, embeddingModel)
    .indexName("classifierIdx")
    .contentFieldName("text")
    .embeddingFieldName("textEmbedding")
    .prefix("classifier:")
    .initializeSchema(true)
    .vectorAlgorithm(RedisVectorStore.Algorithm.FLAT)
    .build()
redisVectorStore.afterPropertiesSet()

### Storing the references as vectors

The references are stored in Redis as vectors. The vectors are created using the embedding model and stored in the Redis Vector Store.

In [35]:
import kotlinx.serialization.json.Json
import org.springframework.ai.document.Document
import java.io.File
import java.util.UUID

val references = Json.decodeFromString<List<String>>(File("resources/filtering-examples.json").readText())

fun createRouteDocument(text: String): Document {
    return Document(
        UUID.randomUUID().toString(),
        text,
        mapOf(
            "text" to text,
        )
    )
}

fun storeRouteDocumentsInRedis(references: List<String>) {
    val documents = references.map { text ->
        createRouteDocument(text)
    }

    redisVectorStore.add(documents)
}

storeRouteDocumentsInRedis(references)

### Creating a Classification Function
Now we'll create a function to classify text using the model.

The function takes a text as input and returns a classification output. The classification output contains the probabilities for each candidate label.


In [36]:
import ai.djl.modality.nlp.translator.ZeroShotClassificationOutput
import org.springframework.ai.vectorstore.SearchRequest

fun breakSentenceIntoClauses(sentence: String): List<String> {
    return sentence.split(Regex("""[!?,.:;()"\[\]{}]+"""))
        .filter { it.isNotBlank() }.map { it.trim() }
}

fun classify(post: String): List<Double> {
    return breakSentenceIntoClauses(post).map { clause ->
        (redisVectorStore.similaritySearch(
            SearchRequest.builder()
                .topK(1)
                .query(clause)
                .build()
        )?.map { it.score ?: 0.0 } ?: emptyList())
    }.flatten()
}

In [37]:
classify("Redis is a great tool for building distributed systems")

[0.7052901387214661]

In [39]:
classify("Redis is a great tool for building applied AI systems because it's a Vector Database")

[0.8183810710906982]

### Creating a Filter Handler
Now we'll create a handler that filters events based on their content.

The handler uses the classification function to determine if a post is software-related.

If the post is not software-related, the handler returns false, which stops the processing of the event.


In [40]:
val filter: (Event) -> Pair<Boolean, String> = { event ->
    if (event.text.isNotBlank() && event.operation != "delete") {
        val classification = classify(event.text)
        if (classification.any { it > 0.75 }) {
            Pair(true, "OK")
        } else {
            Pair(false, "Not a post related to artificial intelligence")
        }
    } else {
        Pair(false, "Text is null or empty")
    }
}

## Storing Filtered Events
In this section, we'll store the filtered events in Redis for further processing.


### Storing Events in Redis
Now we'll create a handler that stores events in Redis. The handler stores the event as a hash in Redis, with the key being the event's URI.


In [41]:
val storeEvent: (Event) -> Pair<Boolean, String> = { event ->
    jedisPooled.hset("post:" + event.uri.replace("at://did:plc:", ""), event.toMap())
    Pair(true, "OK")
}

### Adding Filtered Events to a New Stream
Finally, we'll create a handler that adds filtered events to a new stream. This allows other consumers to process only the filtered events, rather than having to filter the events themselves.


In [42]:
import redis.clients.jedis.params.XAddParams

val addFilteredEventToStream: (Event) -> Pair<Boolean, String> = { event ->
    jedisPooled.xadd(
        "filtered-events",
        XAddParams.xAddParams().id(StreamEntryID.NEW_ENTRY),
        event.toMap()
    )
    Pair(true, "OK")
}

In [43]:
createConsumerGroup("jetstream", "store-example")

## Putting It All Together
Now we'll put all the pieces together to create a complete pipeline for filtering events from the Redis Stream.

In this example we create two consumers that will process the same stream.
- By doing that, we can scale the processing of the events by adding more consumers to the group.
- Redis will make sure that each consumer will receive different messages.


In [46]:
runBlocking {
        listOf(
            async(Dispatchers.IO) {
                consumeStream(
                    streamName = "jetstream",
                    consumerGroup = "store-example",
                    consumer = "store-1",
                    handlers = listOf(
                        filter,
                        printUri,
                        storeEvent,
                        addFilteredEventToStream
                    ),
                    ackFunction = ackFunction(),
                    count = 1,
                    limit = 1000
                )
            },
            async(Dispatchers.IO) {
                consumeStream(
                    streamName = "jetstream",
                    consumerGroup = "store-example",
                    consumer = "store-2", // Different consumer
                    handlers = listOf(
                        filter,
                        printUri,
                        storeEvent,
                        addFilteredEventToStream
                    ),
                    ackFunction = ackFunction(),
                    count = 1,
                    limit = 1000
                )
            }
        ).awaitAll()
}

store-1: Handler stopped processing: Text is null or empty
store-2: Handler stopped processing: Not a post related to artificial intelligence
store-2: Handler stopped processing: Not a post related to artificial intelligence
store-1: Handler stopped processing: Not a post related to artificial intelligence
store-1: Handler stopped processing: Not a post related to artificial intelligence
store-1: Handler stopped processing: Not a post related to artificial intelligence
store-1: Handler stopped processing: Not a post related to artificial intelligence
store-2: Handler stopped processing: Not a post related to artificial intelligence
store-2: Handler stopped processing: Not a post related to artificial intelligence
store-2: Handler stopped processing: Not a post related to artificial intelligence
store-1: Handler stopped processing: Not a post related to artificial intelligence
store-1: Handler stopped processing: Text is null or empty
store-2: Handler stopped processing: Not a post rela

[kotlin.Unit, kotlin.Unit]

## Next Steps
In the next notebook, we'll enrich the filtered events with additional information, such as topic modeling and embeddings for semantic search.
